<a href="https://colab.research.google.com/github/Bombbom/ML4SecurityProject/blob/main/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BiLSTM

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
data = pd.read_pickle("/content/drive/MyDrive/Submit/NT522_Project/dataset_10000_record_fixed_fragment_vectors.pkl")

In [3]:
from __future__ import print_function

import warnings
import numpy as np

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, LSTM, ReLU, Activation, Bidirectional
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adamax
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight, compute_class_weight

#from parser import parameter_parser
from tensorflow.keras.layers import Layer
#from models.loss_draw import LossHistory
from keras import backend as K
from keras import initializers, regularizers, constraints

In [4]:
vectors = np.stack(data.iloc[:, 0].values)
# vectors = vectors.reshape()
labels = data.iloc[:, 1].values

In [5]:
positive_idxs = np.where(labels == 1)[0]
negative_idxs = np.where(labels == 0)[0]
idxs = np.concatenate([positive_idxs, negative_idxs])
undersampled_negative_idxs = np.random.choice(negative_idxs, len(positive_idxs), replace=True)
resampled_idxs = np.concatenate([positive_idxs, undersampled_negative_idxs])
x_train, x_test, y_train, y_test = train_test_split(vectors[resampled_idxs], labels[resampled_idxs],train_size=0.8,random_state=1, stratify=labels[resampled_idxs])

In [6]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [7]:
dropout = 0.2
# y_train = to_categorical(y_train)
lr = 0.002
batch_size = 64
epochs = 10
threshold = 0.5
  
class_weights =  class_weight.compute_class_weight(class_weight='balanced', classes=[0, 1], y=labels)
adamax = Adamax(lr)
model = Sequential()
model.add(Bidirectional(LSTM(300), input_shape=(vectors.shape[1], vectors.shape[2])))
model.add(ReLU())
model.add(Dropout(dropout))
model.add(Dense(300))
model.add(ReLU())
model.add(Dropout(dropout))
model.add(Dense(2, activation='softmax'))

In [8]:
#Tính và lưu trữ kết quả accuracy
model.compile(loss='binary_crossentropy', optimizer=adamax, metrics=['accuracy'])
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/10
125/125 [==============================] - 228s 2s/step - loss: 0.5060 - accuracy: 0.7351
Epoch 2/10
125/125 [==============================] - 215s 2s/step - loss: 0.4155 - accuracy: 0.7958
Epoch 3/10
125/125 [==============================] - 225s 2s/step - loss: 0.3800 - accuracy: 0.8164
Epoch 4/10
125/125 [==============================] - 223s 2s/step - loss: 0.3566 - accuracy: 0.8292
Epoch 5/10
125/125 [==============================] - 198s 2s/step - loss: 0.3383 - accuracy: 0.8413
Epoch 6/10
125/125 [==============================] - 209s 2s/step - loss: 0.3186 - accuracy: 0.8484
Epoch 7/10
125/125 [==============================] - 221s 2s/step - loss: 0.3074 - accuracy: 0.8595
Epoch 8/10
125/125 [==============================] - 217s 2s/step - loss: 0.2946 - accuracy: 0.8625
Epoch 9/10
125/125 [==============================] - 207s 2s/step - loss: 0.2820 - accuracy: 0.8705
Epoch 10/10
125/125 [==============================] - 187s 2s/step - loss: 0.2743 - accura

In [9]:
_, accuracy_test = model.evaluate(x_test, y_test)
_, accuracy_train = model.evaluate(x_train, y_train)
values = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
predictions = (model.predict(x_test, batch_size=batch_size)).round()
tn, fp, fn, tp = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1)).ravel()
FPR=fp / (fp + tn)
FNR= fn / (fn + tp)
recall = tp / (tp + fn)
precision = tp / (tp + fp)
F1=(2 * precision * recall) / (precision + recall)
# storeResults("BLSTM",(accuracy_train*100),(accuracy_test*100),recall*100,FPR*100,precision*100, F1*100)

32/32 [==============================] - 14s 414ms/step


In [10]:
print(tn, fp, fn, tp)

886 114 151 849


In [11]:
print("Accuracy: ", (tp + tn) / (tp + tn + fp + fn))
print('False positive rate(FPR): ', fp / (fp + tn))
print('False negative rate(FNR): ', fn / (fn + tp))
recall = tp / (tp + fn)
print('Recall(TPR): ', recall)
precision = tp / (tp + fp)
print('Precision: ', precision)
print('F1 score: ', (2 * precision * recall) / (precision + recall))

Accuracy:  0.8675
False positive rate(FPR):  0.114
False negative rate(FNR):  0.151
Recall(TPR):  0.849
Precision:  0.881619937694704
F1 score:  0.8650025471217525
